In [1]:
import cv2
import numpy as np
import pickle
import dlib
import mediapipe as mp


### reading videos
capture = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
hand_detector = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils
fpoints = [2, 30, 14]

In [2]:
while True:
    Istrue, frame = capture.read()
    resized = cv2.resize(frame, (int(frame.shape[1]*40/100), int(frame.shape[0]*40/100)), interpolation = cv2.INTER_AREA)
    frame = cv2.flip(resized, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    frame_height, frame_width, _ = frame.shape
    output = hand_detector.process(frame)
    hands = output.multi_hand_landmarks
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    if faces:
        for face in faces:
            landmarks = predictor(gray, face)
        
        rt_dist = landmarks.part(30).x-landmarks.part(2).x
        lt_dist = landmarks.part(14).x-landmarks.part(30).x
        vdiff = rt_dist - lt_dist
        varrow_strt = (landmarks.part(30).x, landmarks.part(30).y)
        # varrow_strt = int(frame_width*0.9),int(frame_height*0.2)
        varrow_end = (varrow_strt[0]+int(vdiff*0.5), varrow_strt[1]+10-int(vdiff*0.03))
        if abs(varrow_strt[0]-varrow_end[0]) < 10:
            vlbl = 'Medial'
        elif varrow_strt[0]-varrow_end[0] >= 10:
            vlbl = 'Right'
        else:
            vlbl = 'Left'
        cv2.arrowedLine(frame, varrow_strt, varrow_end,
                                      (0,0,255), 5)
        cv2.putText(frame, vlbl, (int(frame_width*0.9), int(frame_height*0.25)-5), font, 
                   1, (0,0,255), 3, cv2.LINE_AA)
        cv2.putText(frame, 'face', (int(frame_width*0.75), int(frame_height*0.25)-5), font, 
                   1, (0,0,255), 3, cv2.LINE_AA)
        vcoords = 'X:'+str(landmarks.part(30).x)+' Y:'+str(landmarks.part(30).y)
        cv2.putText(frame, vcoords, (int(frame_width*0.7), int(frame_height*0.33)-5), font, 
                   1, (0,0,255), 3, cv2.LINE_AA)
    
    if hands:
        for hand in hands:
            landmarks = hand.landmark
            for idp, landmark in enumerate(landmarks):
                x = int(landmark.x*frame_width)
                y = int(landmark.y*frame_height)
                if idp == 8:
                    x8 = x
                    y8 = y
                if idp == 5:
                    x5 = x
                    y5 = y
        farrow_strt = (x8,y8)
        farrow_end = (farrow_strt[0]+x8-x5, farrow_strt[1]+y8-y5)
        
        if (farrow_strt[0]-farrow_end[0] < 15) and (farrow_strt[0]-farrow_end[0] > -45):
            flbl = 'Medial'
        elif farrow_strt[0]-farrow_end[0] >= 15:
            flbl = 'Right'
        else:
            flbl = 'Left'
        cv2.arrowedLine(frame, farrow_strt, farrow_end,
                                      (0,255,255), 5)
        cv2.putText(frame, flbl, (int(frame_width*0.9), int(frame_height*0.85)-5), font, 
                   1, (0,255,255), 3, cv2.LINE_AA)
        cv2.putText(frame, 'finger', (int(frame_width*0.75), int(frame_height*0.85)-5), font, 
                   1, (0,255,255), 3, cv2.LINE_AA)
        fcoords = 'X:'+str(x8)+' Y:'+str(y8)
        cv2.putText(frame, fcoords, (int(frame_width*0.7), int(frame_height*0.93)-5), font, 
                   1, (0,255,255), 3, cv2.LINE_AA)
        
    
    if hands and faces:
        if flbl == vlbl:
            fvalign = 'Aligned'            
        else:
            fvalign = 'Misaligned'
        cv2.putText(frame, fvalign, (int(frame_width*0.8), int(frame_height*0.6)), font, 
                   1, (255,0,0), 3, cv2.LINE_AA)
    
    cv2.imshow('video', frame)

    if cv2.waitKey(20) & 0xFF == ord('d'):
        break
        
capture.release()
cv2.destroyAllWindows()  